# A Model for the Human Chromosome
This example shows a simple simulation of a single block copolymer that is a potential model of the Human Chromosome in hoomdd.

The human genome consists of 24 chromosomes with a total size of 3Gb.  Aiden et al. 2014 produced a 1kB resolution (below single gene resolution) Genome Contact Map.  Contact domains (∼185 kb) segregate into six nuclear subcompartments with distinct histone marks.
This can be reproduced phenomenologically with this simple block copolymer model.

from: https://lost-contact.mit.edu/afs//umich.edu/user/j/o/joaander/Public/hoomd-web/doc/page_example_scripts.html

## Initialize
First import hoomd and associated libraries and then initialize.  If no mode is given then the fastest processor cpu or gpu will be selected automatically.

In [1]:
#import hoomd
import hoomd.md
from hoomd import *
from hoomd.deprecated import *
context.initialize("");
#context.initialize("--mode=cpu");
#context.initialize("--mode=gpu");

HOOMD-blue 2.2.4-unknown CUDA (8.0) DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 03/11/2018
Copyright 2009-2017 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
notice(2): NVIDIA driver not installed or is too old, ignoring any GPUs in the system.
HOOMD-blue is running on the CPU


## Import other libraries and define parameters

The human genome has 24 chromasomes but for now we will simulate a single chromosome.

In [24]:
import math
# parameters
phi_P = 0.25
n_poly=1

## Define the polymer
The average length of a human chromosome is 133 Mb

In [25]:
import numpy as np
genome_size=3079843747
NChr=2 #Mb
Nbeads=genome_size/100000
#chromosome=np.linspace(1,Nchromosomes,Nchromosomes,dtype=intp).tolist()
chromosome_size=np.fix(np.r_[8,7.9,6.5,6.2,5.9,5.5,5.2,4.7,4.6,4.4,4.4,4.3,3.7,3.5,3.3,2.9,2.6,2.5,2.1,2.0,1.5,1.6,5.0,1.9]/100.0*Nbeads)
chromosome=list('bcdefghijklmnopqurstuvwxy')
s="["+"]*%d+[".join(map(str, chromosome))+"]*%d" 
t=chromosome_size.astype(int).tolist()
#print t
#type_string='['+''.join(str(e) for e in [val for pair in zip(chromosome,N*']*', chromosome_size.astype(int),N*'+[]') for val in pair])
type_string=['']
for i in range(NChr): type_string += "['%s']*%d" % (chromosome[i],chromosome_size[i]), 
type_string='+'.join(type_string)
type_string=type_string[1:]
print type_string

['b']*2463+['c']*2433


So if we assume that Euchromatin makes up 60% of the genome and heterochromatin makes up 40% then a block copolymer of 10000 beads would have a resolution of approximately 13 kb/bead the average size of a human gene.  This would correspond to approximately 18 beads per enhancer promoter loop.

In [26]:
#type_string='b']*300 + ['c']*400 + ['b']*300
#polymer1 = dict(bond_len=1.2, type=,bond="linear", count=Nchromosomes)
polymer1 = dict(bond_len=1.2, type=type_string,bond="linear", count=n_poly)
# perform some simple math to find the length of the box
N = len(polymer1['type']) * polymer1['count']
# generate the polymer system
separation_radius=dict(b=0.35, c=0.35)
print separation_radius
init.create_random_polymers(box=data.boxdim(volume=10*math.pi * N / (6.0 * phi_P)), polymers=[polymer1],separation=separation_radius,seed=12)


{'c': 0.35, 'b': 0.35}


**ERROR**: No separation radius specified for type [


RuntimeError: Error creating random polymers

## Setup the bonds and force fields

In [5]:
# force field setup
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=330.0, r0=0.84)
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(nlist=nl, r_cut=3.0)
lj.pair_coeff.set('b', 'b', epsilon=1.0, sigma=1.0, alpha=0.0, r_cut=2**(1.0/6.0))
lj.pair_coeff.set('b', 'c', epsilon=1.0, sigma=1.0, alpha=0.0, r_cut=2**(1.0/6.0))
lj.pair_coeff.set('c', 'c', epsilon=1.0, sigma=1.0, alpha=1.0)
lj.set_params(mode='shift')
                  


## Special Pairs will create our Contact Domains (not working yet)

In [6]:
#CTCF = hoomd.md.special_pair()
#lj_ctcf = hoomd.md.special_pair.lj(name="CTCF")
#lj_ctcf.pair_coeff.set('A','A', epsilon=1.0, sigma=1.0, r_cut=3)
#lj_ctcf.set_params(mode='shift')

## Integrate the simulation

In [ ]:
all = group.all()
# integrate NVT for a bunch of time steps
hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.nvt(group=all, kT=1.2, tau=0.5)
#hoomd.md.integrate.brownian(group=all, kT=1.2, seed=1)

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

In [ ]:
#hoomd.dump.gsd(filename="hum_gen_24x1000.gsd", overwrite=True, period=None, group=group.all(), time_step=0)
hoomd.dump.gsd("gsd/24xchromosomes_10000_0.gsd", period=None, group=all, overwrite=True, time_step=0);
# setup the IMD server
hoomd.analyze.imd(port=54321, period=100)
# run a very long time so the simulation can be watched in VMD
run(1e6)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 2
notice(2): Particles with 2 exclusions             : 998
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
notice(2): analyze.imd: listening on port 54321
Time 00:00:10 | Step 15086 / 1000000 | TPS 1508.57 | ETA 00:10:52
notice(2): analyze.imd: accepted connection
Time 00:00:20 | Step 26046 / 1000000 | TPS 1095.99 | ETA 00:14:48
Time 00:00:30 | Step 36001 / 1000000 | TPS 995.484 | ETA 00:16:08
Time 00:00:40 | Step 45919 / 1000000 | TPS 991.612 | ETA 00:16:02
Time 00:00:50 | Step 56826 / 1000000 | TPS 1090.67 

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

While running must execute the following command in a terminal:

In [ ]:
#vmd -e imd6.vmd
!cat imd6.vmd

![](snapshots/chromosome_10kb.png)